---
title: "Classification"
format:
  pdf:
    toc: true
    colorlinks: true
    
  html:
    code-fold: false
jupyter: python3
author: "Matthew Loh"
---

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import (
    mean_squared_error,
    r2_score,
    accuracy_score,
    classification_report,
)
from sklearn.model_selection import GridSearchCV


class MLBackend:
    def __init__(self, problem_type):
        self.problem_type = problem_type  # 'regression' or 'classification'
        self.data = None
        self.X = None
        self.y = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.model = None
        self.preprocessor = None

    def load_data(self, file_path):
        # Load your dataset here
        self.data = pd.read_csv(file_path)  # Adjust based on your data format
        print("Data loaded. Shape:", self.data.shape)

    def preprocess_data(self):
        # Identify features and target
        self.X = self.data.drop(
            "target_column", axis=1
        )  # Replace 'target_column' with your actual target column name
        self.y = self.data["target_column"]

        # Split the data
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, test_size=0.2, random_state=42
        )

        # Create preprocessing steps
        numeric_features = self.X.select_dtypes(include=["int64", "float64"]).columns
        categorical_features = self.X.select_dtypes(include=["object"]).columns

        numeric_transformer = Pipeline(
            steps=[
                ("imputer", SimpleImputer(strategy="median")),
                ("scaler", StandardScaler()),
            ]
        )

        categorical_transformer = Pipeline(
            steps=[
                ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
                # Add one-hot encoding or other categorical encoding here
            ]
        )

        self.preprocessor = ColumnTransformer(
            transformers=[
                ("num", numeric_transformer, numeric_features),
                ("cat", categorical_transformer, categorical_features),
            ]
        )

        # Fit and transform the training data
        self.X_train = self.preprocessor.fit_transform(self.X_train)
        self.X_test = self.preprocessor.transform(self.X_test)

        print("Data preprocessed.")

    def train_model(self):
        if self.problem_type == "regression":
            models = {
                "Linear Regression": LinearRegression(),
                "Random Forest Regression": RandomForestRegressor(random_state=42),
            }
        else:  # classification
            models = {
                "Logistic Regression": LogisticRegression(random_state=42),
                "Random Forest Classification": RandomForestClassifier(random_state=42),
            }

        best_score = float("-inf")
        for name, model in models.items():
            model.fit(self.X_train, self.y_train)
            if self.problem_type == "regression":
                score = r2_score(self.y_test, model.predict(self.X_test))
            else:
                score = accuracy_score(self.y_test, model.predict(self.X_test))

            print(f"{name} - Score: {score}")
            if score > best_score:
                best_score = score
                self.model = model

        print(f"Best model: {type(self.model).__name__}")

    def tune_model(self):
        if isinstance(self.model, (RandomForestRegressor, RandomForestClassifier)):
            param_grid = {
                "n_estimators": [100, 200, 300],
                "max_depth": [None, 10, 20, 30],
                "min_samples_split": [2, 5, 10],
                "min_samples_leaf": [1, 2, 4],
            }
        elif isinstance(self.model, LogisticRegression):
            param_grid = {
                "C": [0.1, 1, 10],
                "penalty": ["l1", "l2"],
                "solver": ["liblinear", "saga"],
            }
        else:  # LinearRegression
            print("Linear Regression doesn't have hyperparameters to tune.")
            return

        grid_search = GridSearchCV(
            self.model,
            param_grid,
            cv=5,
            scoring="r2" if self.problem_type == "regression" else "accuracy",
        )
        grid_search.fit(self.X_train, self.y_train)
        self.model = grid_search.best_estimator_
        print("Model tuned. Best parameters:", grid_search.best_params_)

    def evaluate_model(self):
        y_pred = self.model.predict(self.X_test)
        if self.problem_type == "regression":
            mse = mean_squared_error(self.y_test, y_pred)
            r2 = r2_score(self.y_test, y_pred)
            print(f"Mean Squared Error: {mse}")
            print(f"R2 Score: {r2}")
        else:
            accuracy = accuracy_score(self.y_test, y_pred)
            print(f"Accuracy: {accuracy}")
            print("Classification Report:")
            print(classification_report(self.y_test, y_pred))


# Usage example
if __name__ == "__main__":
    # For regression
    regression_backend = MLBackend("regression")
    regression_backend.load_data("path_to_regression_data.csv")
    regression_backend.preprocess_data()
    regression_backend.train_model()
    regression_backend.tune_model()
    regression_backend.evaluate_model()

    # For classification
    classification_backend = MLBackend("classification")
    classification_backend.load_data("path_to_classification_data.csv")
    classification_backend.preprocess_data()
    classification_backend.train_model()
    classification_backend.tune_model()
    classification_backend.evaluate_model()